### Word2vec 모델을 학습시키고 사용해보는 것이 목표
* 나무위키 덤프 활용 - 정제 - 텍스트로 된 순수 문장 얻기
* 형태소 분석 - 형태소 분류
* 말뭉치(Corpus) 데이터 준비 
* 말뭉치를 활용해서 Word2vec 모델 학습
* Word2vec 모델 활용

In [1]:
import ijson
import gensim
import re

In [2]:
input_filename = "/Users/byeon/Downloads/namuwiki180326/namuwiki_20180326.json"
output_filename = "/Users/byeon/Downloads/namuwiki180326/namuwiki_20180326_mini.txt"

In [3]:
input_file = open(input_filename, 'r', encoding='utf-8')
texts = ijson.items(input_filename, 'item.text')

In [22]:
for some in zip(range(3), texts):
    print(some)

(0, '#redirect 느낌표\n')
(1, '[[파일:3444050440.jpg]]\n([[신 세계수의 미궁 2]]에서 뜬 !!아앗!!)\n{{{+1 ！！ああっと！！ }}}\n\n[[세계수의 미궁 시리즈]]에 전통으로 등장하는 대사. [[세계수의 미궁 2 제왕의 성배|2편 제왕의 성배]]부터 등장했으며, 훌륭한 [[사망 플래그]]의 예시이다.\n\n세계수의 모험가들이 탐험하는 던전인 수해의 구석구석에는 채취/벌채/채굴 포인트가 있으며, 이를 위한 채집 스킬에 투자하면 제한된 채집 기회에 보다 큰 이득을 챙길 수 있다. 그러나 분배할 수 있는 스킬 포인트는 한정된 만큼 채집 스킬에 투자하는 만큼 전투 스킬 레벨은 낮아지게 된다.\n\n 1. 채집용 캐릭터들로 이루어진 약한 파티(ex: [[레인저(세계수의 미궁 2)|레인저]] 5명)가 수해에 입장한다.\n 1. 필드 전투를 회피하면서 채집 포인트에 도착해 열심히 아이템을 캐는 중에...\n 1. \'\'\'!!아앗!!\'\'\' ~~라플레시아가 나타났다!~~\n \'\'\'이때 등장하는 것은 [[FOE(세계수의 미궁 시리즈)|FOE]]는 아닌 일단 필드 졸개지만, 훨씬 위 층에 등장하는 강력한 졸개이며 선 턴을 빼앗긴다!\'\'\'\n 1. \'\'\'[[으앙 죽음|떡잎]]\'\'\'(hage)\n\n작품마다 !!아앗!!의 세세한 모습은 다르다. 그 악랄함은 첫 등장한 작품이자 시리즈 중에서도 불친절하기로 정평이 난 2편이 절정이었는데, 그야말로 위의 !!아앗!! 시퀀스 그대로, 묻지도 따지지도 않고 채집할 때마다 일정 확률로 \'\'\'강제로\'\'\' 전투에 돌입해야 했다. 게다가 이럴 때 쓰라고 있는 레인저의 스킬 \'위험감지(중간 확률로 적의 선제공격을 무효화)\'는 작동하지 않았다!\n[[세계수의 미궁 3 성해의 내방자|3편]], [[세계수의 미궁 4 전승의 거신|4편]]에는 숨통이 트이게도 채집 중 낮은 확률로 "좋은 아이템을 얻을 수 있을 것 같지만... 주변에서 몬스터들의 기척이 느껴진다."는

In [6]:
chinese = re.compile(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', re.UNICODE)
japanese = re.compile(u'[\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uff9f\u4e00-\u9faf\u3400-\u4dbf]', re.UNICODE)

# hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
# hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
# result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거


def strip_wiki_literal(text):               
    text = re.sub(r"\{\{\{#\!html[^\}]*\}\}\}", '', text, flags=re.IGNORECASE|re.MULTILINE|re.DOTALL) # remove html
    text = re.sub(r"#redirect .*", '', text, flags=re.IGNORECASE) # remove redirect
    text = re.sub(r"\[\[분류:.*", '', text) # remove 분류
    text = re.sub(r"\[\[파일:.*", '', text) # remove 파일
    text = re.sub(r"\* 상위 문서 ?:.*", '', text) # remove 상위문서        
    text = re.sub(r"\[youtube\(\w+\)\]", '', text, flags=re.IGNORECASE) # remove youtube
    text = re.sub(r"\[include\(([^\]|]*)(\|[^]]*)?\]", r'\1', text, flags=re.IGNORECASE) # remove include
    text = re.sub(r"\[\[(?:[^\]|]*\|)?([^\]|]+)\]\]", r'\1', text) # remove link
    text = re.sub(r"\[\*([^\]]*)\]", '', text) # remove 각주
    text = re.sub(r"\{\{\{([^\ }|]*) ([^\}|]*)\}\}\}", r'\2', text) # remove text color/size
    text = re.sub(r"'''([^']*)'''", r'\1', text) # remove text bold
    text = re.sub(r"(~~|--)([^']*)(~~|--)", '', text) # remove strike-through
    text = re.sub(r"\|\|(.*)\|\|", '', text) # remove table
    text = re.sub(r"\n\s*\n*", '\n', text) # remove empty line                       
    text = chinese.sub('', text) # remove chinese
    text = japanese.sub('', text) # remove japanese
    return text

In [26]:
strip_wiki_literal(" XX는 더 이상합니다[* 어색한 상황을 수습해보려는 변명이 더 안 좋은 결과를 낼 때 나오는 말.] ||\n\n[목차]\n\n== 소개 ==\n--[[나는 킬러다|2년 뒤에는 사위가 장인을 죽이려 한다]].-")

' XX는 더 이상합니다 ||\n[목차]\n== 소개 ==\n--2년 뒤에는 사위가 장인을 죽이려 한다.-'

In [28]:
for index, text in zip(range(3), texts):
    print(strip_wiki_literal(text))

틀:링크시 주의, 링크=[[\\\\#FairyJoke])]
[목차]
== 개요 ==
사운드 볼텍스 시리즈에 수록된 동방 프로젝트 어레인지 곡이며, 리믹서는 uno(IOSYS). \#Fairy_dancing_in_lake의 후속곡이기도 하다. 원곡은 동방홍마향 2면 필드곡인 '르네이트 엘프()'이며, 쥬크 하우스의 영향을 받아 리믹스했다고 리믹서 본인이 밝힌 바 있다.
동 작곡가가 DJ Laugh 명의로 그루브 코스터에 제공한 Stardust Vox와 전개, 음색이 많이 닮았다.
제 11회 하쿠레이 신사 예대제에서 발매된 IOSYS의 앨범 Prank Masters!에 수록되었다.
== 사운드 볼텍스에 수록 ==
* 곡 목록으로 돌아가기
* SKILL ANALYZER 수록
* EXHAUST : Skill Level 10(2014.7.4 ~ 2014.11.20), Skill Level 10 C코스(2014.11.20 ~ 2014.12.26)
자켓은 하쿠레이 신사 예대제 11회에 발매된 Prank Masters!의 표지를 그대로 썼다. 자켓의 캐릭터는 대요정과 치르노.
=== 채보 상세 ===
[youtube(HJM9vrD8Ngw, width=420, height=600)]
EXH 패턴 PERFECT 영상
좌우 노브 분리 영상
이미 2014년 5월 동방 예대제에서 \#Fairy_dancing_in_lake의 후속곡이 수록될 것이라는 정보가 밝혀졌었고, 그보다 훨씬 악랄한 패턴을 보여주겠다는 예고가 떴다. 그리고 수록된 것은 대다수가 예상한 Hirayasu Matsudo 제작의 고난이도 노브곡. Hirayasu Matsudo가 처음으로 단독 제작한 레벨 15.
페어리 댄싱의 물수제비를 꼬아 놓은데다가 중간중간 보이는 HYENA의 고속 직각노브, GAMBOL(dfk SLC rmx) INF의 꽈배기 노브 등등 기존의 노브곡들의 대표 패턴들이 한데 섞여 나온다. 그리고 기습적으로 섞여나오는 BT, FX 트릴 때문에 여타 다른 Hirayasu Matsudo의 노브곡에 비해 니

In [4]:
input_file = open(input_filename, 'r', encoding='utf-8')
texts = ijson.items(input_file, 'item.text')

In [7]:
%%time

item_limit = 10000
minimum_length = 500

with open(output_filename, 'w', encoding='utf-8') as output_file:
    for index, text in zip(range(item_limit), texts):
        if (len(text) > minimum_length):
            try:
                a_line = strip_wiki_literal(text)
                output_file.write(a_line + '\n')
            except UnicodeEncodeError as e:
                print("UnicodeEncodeError ({0}) : {1}".format(e, text))

CPU times: user 54.1 s, sys: 13 s, total: 1min 7s
Wall time: 1min 9s


In [8]:
import konlpy

In [9]:
from konlpy.tag import Twitter

In [10]:
result = Twitter().pos("나무위키 말뭉치를 만들어보자")
for pos in result:
    print(pos[0]+ ' ' + pos[1])

나무 Noun
위키 Noun
말뭉치 Noun
를 Josa
만들어 Verb
보자 Verb


In [11]:
tagger = Twitter()

In [12]:
def flat(content):
    return ["{}/{}".format(word, tag) for word, tag in tagger.pos(content)]

In [13]:
tagged = flat("나무위키 말뭉치를 만들어보자")

In [14]:
input_filename = "/Users/byeon/Downloads/namuwiki180326/namuwiki_20180326_mini.txt"
output_filename = "/Users/byeon/Downloads/namuwiki180326/namuwiki_20180326_mini_pos_tagged_corpus.txt"

In [15]:
%%time
with open(output_filename, 'w', encoding='utf-8') as output_file:
    for line in open(input_filename, 'r', encoding='utf-8'):
        for sentence in line.split('.'):
            tagged = flat(sentence)
            if len(tagged) > 1:
                a_line = ' '.join(tagged)
                output_file.write(a_line + '\n')

CPU times: user 8min 7s, sys: 7.09 s, total: 8min 14s
Wall time: 8min 19s


In [16]:
from gensim import corpora, similarities
from gensim.models import Word2Vec

import os
import multiprocessing

input_filename = '/Users/byeon/Downloads/namuwiki180326/namuwiki_20180326_mini_pos_tagged_corpus.txt'
model_path = './model'

In [17]:
%%time
class SentenceReader(object):
    def __init__(self, input_filename):
        self.input_filename = input_filename
 
    def __iter__(self):
        for line in open(input_filename):
            yield line.split(' ')

sentences_vocab = SentenceReader(input_filename) # a memory-friendly iterator
sentences_train = SentenceReader(input_filename) # a memory-friendly iterator

config = {
    'min_count': 10,  # 등장 횟수가 10 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 10,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
}
word2vec_model = Word2Vec(**config)

CPU times: user 107 µs, sys: 6 µs, total: 113 µs
Wall time: 118 µs


In [18]:
%%time
token_count = sum([len(sentence) for sentence in sentences_vocab])
print(token_count)

word2vec_model.build_vocab(sentences_vocab)
word2vec_model.train(sentences_train, total_examples = token_count, epochs=word2vec_model.iter)

10103580


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


CPU times: user 24min 51s, sys: 15.8 s, total: 25min 7s
Wall time: 3min 57s


In [25]:
word2vec_model.wv.most_similar(["서울/Noun"])

[('부산/Noun', 0.5920711159706116),
 ('장충/Noun', 0.575017511844635),
 ('여의도/Noun', 0.5566101670265198),
 ('영등포구/Noun', 0.5554679036140442),
 ('종로/Noun', 0.5489655137062073),
 ('영등포/Noun', 0.547404408454895),
 ('압구정/Noun', 0.5453184843063354),
 ('인천/Noun', 0.5452240705490112),
 ('상암동/Noun', 0.5448856353759766),
 ('고법/Noun', 0.5424453616142273)]

In [26]:
word2vec_model.wv.most_similar(positive = ['서울/Noun', '미국/Noun'], negative=['한국/Noun'])

[('제네바/Noun', 0.4128422439098358),
 ('뮌헨/Noun', 0.3984510600566864),
 ('컬럼비아/Noun', 0.39666545391082764),
 ('지방법원/Noun', 0.3959895670413971),
 ('건설/Noun\n', 0.3912340998649597),
 ('국회의사당/Noun', 0.3874393701553345),
 ('영등포/Noun', 0.3866955041885376),
 ('상트페테르부르크/Noun', 0.38594767451286316),
 ('영등포구/Noun', 0.3842099905014038),
 ('장충/Noun', 0.38419654965400696)]

In [28]:
word2vec_model.save(model_path)